# Recolecciòn de datos

La data se obtuvo del repositorio UCI de Machine Learning Concrete Compressive Strength Data Set, de los cuales se uso una muestra de 139 para los datos correctos y 10 para los incorrectos modificando manualmente 
Link https://archive.ics.uci.edu/dataset/165/concrete+compressive+strength
//

**Diccionario de datos (basado en el conjunto real)**
- NombreTipoConcreto (string): Clasificación ficticia del tipo de mezcla, derivado de los ingredientes (por ejemplo, “AltaSlag_540”).
- Origen (string): Procedencia o etiqueta de la muestra, como “LaboratorioUCI”.
- Edad (int): Edad del concreto en días (entre 1 y 365) 
- CantidadCemento (int): Cantidad de cemento en kg por m³ de mezcla (valor típico entero) 
- Resistencia (double): Resistencia a compresión medida en MPa (valor con decimales

# Flujo Auto loader

In [0]:
%sql
CREATE OR REPLACE TABLE dmc_01.default.Cemento(
  TipoConcreto  STRING,
  Origen STRING,
  Edad INT,
  CantidadCemento INT,
  Resistencia DOUBLE
)
USING DELTA;

# Creamos el esquema con el StructType

In [0]:
from pyspark.sql.types import StructType, StringType, DoubleType, IntegerType

cemento_schema = StructType() \
    .add("TipoConcreto", StringType()) \
    .add("Origen", StringType()) \
    .add("Edad", IntegerType()) \
    .add("CantidadCemento", IntegerType()) \
    .add("Resistencia", DoubleType()) 

In [0]:
from pyspark.sql.functions import col

df_cemento = (spark.read
    .format("json")
    .schema(cemento_schema)
    .load("/Volumes/dmc_01/default/volumen_01/tareas/tarea_01/input")
    .withColumn("archivo_origen", col("_metadata.file_path"))
    )    

In [0]:
display(df_cemento)

In [0]:
from pyspark.sql.types import StructType, StringType, IntegerType, DoubleType, TimestampType

cemento_schema = (
    StructType()
    .add("TipoConcreto", StringType())
    .add("Origen", StringType())
    .add("Edad", IntegerType())
    .add("CantidadCemento", IntegerType())
    .add("Resistencia", DoubleType())
    .add("_rescued_data", StringType())
)


# Agregar una columna de auditoría 
En la ruta de archivo de origen (_metadata.file_path).

In [0]:
from pyspark.sql.functions import col

df_cemento = (
    spark.read
    .format("json")
    .schema(cemento_schema)
    .option("columnNameOfCorruptRecord", "_rescued_data")
    .load("/Volumes/dmc_01/default/volumen_01/tareas/tarea_01/input")
    .withColumn("archivo_origen", col("_metadata.file_path"))
)

# badRecordsPath
Configurar un badRecordsPath y capturar información de registros inválidos.

In [0]:
from pyspark.sql.functions import col
dbutils.fs.rm("/Volumes/dmc_01/default/volumen_01/tareas/tarea_01/bad", True)
df_cemento = (
    spark.read
    .format("json")
    .schema(cemento_schema)
    #.option("columnNameOfCorruptRecord", "_rescued_data")
    .option("badRecordsPath", "/Volumes/dmc_01/default/volumen_01/tareas/tarea_01/")
    .load("/Volumes/dmc_01/default/volumen_01/tareas/tarea_01/input")
    .withColumn("archivo_origen", col("_metadata").getField("file_path"))
)

In [0]:
display(df_cemento)

# Tabla Delta de auditoría
Guardar los registros inválidos en una tabla Delta de auditoría usando saveAsTable().

In [0]:
from pyspark.sql.functions import col

try:
    df_validos = df_cemento.filter(col("_rescued_data").isNull())
    df_invalidos = df_cemento.filter(col("_rescued_data").isNotNull())

    df_validos.write.format("delta").mode("overwrite").saveAsTable("dmc_01.default.concreto")
    df_invalidos.write.format("delta").mode("overwrite").saveAsTable("dmc_01.default.concreto_errores")

    print("Escritura realizada con éxito.")

except Exception as e:
    print(f"Error durante la escritura: {str(e)}")

In [0]:
%sql
select * from dmc_01.default.concreto

In [0]:
%sql
select * from dmc_01.default.concreto_errores